# CZĘŚĆ 1 - OPTYMALIZACJA

## OPIS ZESTAWU DANYCH
Dane składają się z informacji o przylotach i odlotach wszystkich lotów komercyjnych w USA od października 1987 do kwietnia 2008 – przede wszystkim o ich opóźnieniach. \
Zbiór danych jest bardzo duży (120mln rekordów, 12GB danych) – na potrzeby projektu wykorzystamy jedynie dane z roku 2007 co ograniczy rozmiar przetwarzanych danych.

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import pyarrow as pa
from matplotlib.pyplot import figure
from matplotlib import pyplot as plt
import statistics
from sklearn.impute import KNNImputer
import seaborn as sns
import glob
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnull, when, count, col, hour, mean, lit, stddev,abs
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorAssembler
from sklearn.metrics import confusion_matrix
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, DecisionTreeRegressor
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier, MultilayerPerceptronClassifier


#### OPIS KLAS
W sumie klas jest 29, opisują one następujące informacje:
* rok
* miesiąc
* dzień miesiąca
* dzień tygodnia
* rzeczywisty czas odlotu
* zaplanowany czas odlotu
* rzeczywisty czas przylotu
* zaplanowany czas przylotu
* kod przewoźnika
* numer lotu
* numer ogonowy samolotu
* całkowity czas lotu w minutach
* rzeczywisty czas lotu
* całkowity czas w powietrzu
* opóźnienie lotu w minutach
* miejsce startu
* miejsce docelowe
* odległość w milach
* dane dotyczące przyjazdu taksówki
* informacje o tym czy lot był anulowany
* powód anulowania (pogoda, przewoźnik, ochrona, NAS)
* przekierowanie (tak/nie)
* opóźnienie przewoźnika w minutach
* opóźnienie pogodowe w minutach
* opóźnienie NAS w minutach
* opóźnienie z powodów bezpieczeństwa w minutach
* sumaryczne opóźnienie w minutach

Celem projektu jest przewidywanie sumarycznego opóźnienia samolotu - zmienna objaśniana - na podstawie podzbioru pozostałych kolumn (zmiennych objaśniających), które wybierzemy na podstawie dalszej analizy.

In [ ]:
#TO DO dodać czytanie z kilku plików na raz
# data = pd.concat([pd.read_csv(f) for f in glob.glob('./data/*.csv')], ignore_index = True)
data = pd.read_parquet('2007.gzip')
with pd.option_context('display.float_format', '{:.2f}'.format, 'display.max_rows', None, 'display.max_columns', None):

display(data.head())

In [ ]:
for column in ['DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime']:
    data[column] = data[column] // 100 + (data[column] % 100) / 60

  Jak widać w powyższej tabeli, niektóre kolumny zawierają dane tekstowe - UniqueCarrier, TailNum, Origin, Dest i CancellationCode. \
Z racji tego, że w projekcie chcielibyśmy się skupić na powiązaniach między opóźnieniami/odwołaniami lotów, a momentem ich odbywania, część danych będzie nam zbędna. Dlatego też zdecydowaliśmy się na usunięcie kolumn:
- UniqueCarrier - indywidualny kod przewoźnika
- TailNum - numer ogonowy
- Origin - miejsce rozpoczęcia podróży
- Dest - cel podróży
- CancellationCode - kod odwołania

Ponadto usuwamy także poniższe kolumny:
- FlightNum - ponieważ pełni on rolę numeru ID, więc nie będzie miało większego sensu uwzględnianie go w modelu.
- TaxiIn, TaxiOut - ponieważ dane na temat taksówki naszym zdaniem nie mają wpływu na opóźnienie/odwołanie lotu.

In [ ]:
data.drop('UniqueCarrier', inplace=True, axis =1)
data.drop('TailNum', inplace=True, axis =1)
data.drop('Origin', inplace=True, axis =1)
data.drop('Dest', inplace=True, axis =1)
data.drop('CancellationCode', inplace=True, axis =1)
data.drop('FlightNum', inplace=True, axis =1)
data.drop('TaxiIn', inplace=True, axis =1)
data.drop('TaxiOut', inplace=True, axis =1)

In [ ]:
with pd.option_context('display.float_format', '{:.2f}'.format, 'display.max_rows', None, 'display.max_columns', None):

display(data.describe())

Jak widać w powyższej tabeli dane obejmują okres od stycznia do czerwca 2007 i są dość równo rozłożone w tym okresie - średni miesiąc to między marcem a kwietniem, dni miesiąca oraz dni tygodnia mają równo rozłożone kwartyle. \



## WIZUALIZACJA DANYCH

### BOXPLOTY

In [ ]:
columns_for_visualisation = [column for column in data.columns if column != 'Year']

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(20, 20))
axes = axes.flatten()
for i, column in enumerate(columns_for_visualisation):
    sns.boxplot(data[column], ax=axes[i], orient='v')
    axes[i].set_title(column, fontsize=15)
    axes[i].set_ylabel('')
    axes[i].set_xticks([])
plt.show()

In [ ]:
fig, axes = plt.subplots(4,4, figsize=(10,10))
for c, i in zip(data.columns, range(0,len(data.columns))):
    a = data.boxplot(column=c, ax=axes.flatten()[i])
plt.tight_layout()
plt.show()

#### TO DO: Dodać komentarz dotyczący boxplotów

### HISTOGRAMY

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(20, 10))
axes = axes.flatten()
for i, column in enumerate(columns_for_visualisation):
    sns.histplot(data[column], ax=axes[i], bins=20)
plt.show()

In [ ]:
fig, axes = plt.subplots(4, 6, figsize=(10,10))
for c, i in zip(data.columns, range(0,24)):
    a = data.hist(column=c, ax=axes.flatten()[i], log= True)
plt.tight_layout()
plt.show()

#### TO DO: Dodać komentarz dotyczący histogramów

### SCATTER PLOT

In [ ]:
columns_for_scatter = ['DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime']

In [ ]:
#scatter plot matrix
sns.pairplot(data, vars=columns_for_scatter)

#### TO DO: Dodać komentarz dotyczący scatterplot

## PROBLEMY Z DANYMI - DANE BRAKUJĄCE, NIEPRAWIDŁOWE, ODSTAJĄCE

### SPRAWDZENIE POPRAWNOŚCI TYPÓW DANYCH

In [ ]:
table = pa.Table.from_pandas(data)
print(table.schema)

Jak widać wyżej - wszystkie dane występują w poprawnym formacie.

### NAPRAWA WIERSZY Z PUSTYMI DANYMI

In [ ]:
# Obliczenie ilosci pustych danych
# Obliczenie ilosci pustych danych
np.sum(data.isna())

In [ ]:
sizeBeforeDeleteNull= data.count()
dataWithoutNull = data.dropna()
sizeAfterDeleteNull =  dataWithoutNull.count()
print("Usunieto: ", sizeBeforeDeleteNull - sizeAfterDeleteNull)

print("Percent od reduced rows: ", 100*sum(sizeBeforeDeleteNull - sizeAfterDeleteNull)/sum(sizeBeforeDeleteNull))

In [ ]:
### TO DO: ZMIENIC SPOSÓB RADZENIA SOBIE Z PUSTYMI DANYMI
### TO DO: Dodać komentarz dotyczący danych pustych

### USUWANIE DANYCH ODSTAJĄCYCH

In [ ]:
sizeBefore = np.shape(data)[0]
for col in data.columns:
    data = data[np.abs(data[col]-data[col].mean()) <= (3*data[col].std())]
sizeAfter =  np.shape(data)[0]
print("Count of reduced rows: ", sizeBefore - sizeAfter)
print("Percent od reduced rows: ", 100*(sizeBefore - sizeAfter)/sizeBefore)

#### TO DO: Dodać komentarz dotyczący danych odstających

# KORELACJE

In [ ]:
# sns.pairplot(data)

In [ ]:
plt.figure(figsize=(20, 20), dpi=80)
corrMatrix = data.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

#### TO DO: Dodać komentarz dotyczący korelacji między danymi, ew dodać pairploty do wybranych danych

In [ ]:
### TO DO: "Normalizacja danych (przedstawić wyniki min-max i standaryzacji). Zastanowić się nad zakresem skalowania danych"


## REDUKCJA WYMIAROWOŚCI

In [ ]:
### TO DO: Genetyczna optymalizacja cech

In [ ]:
# Genetyczna optymalizacja cech - NA RAZIE SAMA SELEKCJA NA PODSTAWIE KORELACJI
pandasDF = data.filter(["DayOfWeek", "DayofMonth", "Distance", "DepTime", "Cancelled", "Diverted", "LateAircraftDelay"], axis = 1)

# PRZYGOTOWANIE MODELI REGRESJI

In [ ]:
spark = SparkSession.builder.master("local").appName("Project").getOrCreate()
sparkDF=spark.createDataFrame(pandasDF)
sparkDF.printSchema()

### PODZIAŁ DANYCH

In [ ]:

vector = VectorAssembler(inputCols = ['DayOfWeek', 'DayofMonth', 'Distance', 'DepTime', 'Cancelled', 'Diverted'
], outputCol = 'features')
vectorData = vector.transform(sparkDF).select(['features', 'LateAircraftDelay'])
vectorData.plot.bar(x = 'parameter', y= 'Coefficients')
plt.show()

vectorData.show(5)

In [ ]:
train_df, test_df = sparkDF.randomSplit([0.7, 0.3])
print("train.rows: ", train_df.count())
print("test.rows: ", test_df.count())

In [ ]:
### MIARY SKUTECZNOŚCI

evaluatorRMSE = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
evaluatorR2 = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="r2")


def effectivenessMeasures(model, predictions):
    print("Coefficients: " + str(model.coefficients))
    print("Intercept: " + str(model.intercept))
    print(predictions.show(5))
    rmse = evaluatorRMSE.evaluate(predictions)
    r2 = evaluatorR2.evaluate(predictions)
    print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
    print("R2 on test data = %g" % r2)


### LOGISTIC REGRESSION

In [ ]:

linear_reg = LinearRegression(featuresCol = 'features', labelCol='label', maxIter=20, regParam=0.3, elasticNetParam=0.8)
linear_model = linear_reg.fit(train_df)
linear_predictions = linear_model.transform(test_df)
effectivenessMeasures(linear_model, linear_predictions)

## RANDOM FOREST regressor

In [ ]:
rf = RandomForestRegressor(featuresCol = 'features', labelCol='label')

rf_model = rf.fit(train_df)
rf_predictions = rf_model.transform(test_df)

effectivenessMeasures(rf_model, rf_predictions)

## DECISION TREE REGRESSOR

In [ ]:
dt = DecisionTreeRegressor(featuresCol = 'features', labelCol='label')
dt_model = dt.fit(train_df)
dt_predictions = dt_model.transform(test_df)

effectivenessMeasures(dt_model, dt_predictions)

# KLASYFIKACJA

### MIARY SKUTECZNOŚCI - KLASYFIKACJA

In [ ]:

def effectivenessMeasuresClassifier(predictions):
    predictionCol = "prediction"
    labelCol = "Odwołane"
    acc=MulticlassClassificationEvaluator(predictionCol=predictionCol, labelCol=labelCol, metricName='accuracy').evaluate(predictions)
    print("Prediction Accuracy: ", acc)

    f1=MulticlassClassificationEvaluator(predictionCol=predictionCol, labelCol=labelCol, metricName='f1').evaluate(predictions)
    print("F1: ", f1)

    precision=MulticlassClassificationEvaluator(predictionCol=predictionCol, labelCol=labelCol, metricName='weightedPrecision').evaluate(predictions)
    print("Precision: ", precision)

    recall=MulticlassClassificationEvaluator(predictionCol=predictionCol, labelCol=labelCol, metricName='weightedRecall').evaluate(predictions)
    print("Recall: ", recall)

    y_pred=predictions.select(predictionCol).collect()
    y_orig=predictions.select(labelCol).collect()

    cm = confusion_matrix(y_orig, y_pred)
    print("Confusion Matrix:")
    print(cm)

    return acc, f1, precision, recall


### DECISION TREE CLASSIFIER

In [ ]:

(trainingData, testData) = vectorData.randomSplit([0.7, 0.3])
dtc = DecisionTreeClassifier(featuresCol="features", labelCol="SoilType").fit(trainingData)
pred = dtc.transform(testData)
# pred.show()
eff_dtc = effectivenessMeasuresClassifier(pred)

### RANDOM FOREST CLASSIFIER

In [ ]:
rfc = RandomForestClassifier(labelCol="SoilType", featuresCol="features", numTrees=10).fit(trainingData)
pred = rfc.transform(testData)
eff_rfc = effectivenessMeasuresClassifier(pred)

### MULTILAYER PERCEPTRON CLASSIFIER

In [ ]:
mpc = MultilayerPerceptronClassifier(labelCol="SoilType", featuresCol="features", numTrees=10).fit(trainingData)
pred = mpc.transform(testData)
eff_mpc = effectivenessMeasuresClassifier(pred)

### GBT CLASSIFIER

In [ ]:
gbt = GBTClassifier(labelCol="SoilType", featuresCol="features", numTrees=10).fit(trainingData)
pred = gbt.transform(testData)
eff_gbt = effectivenessMeasuresClassifier(pred)

### ENSAMBLE CLASSIFICATOR - EXTRA TREES CLASSIFIER

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
etr = ExtraTreesClassifier(n_estimators = 100, criterion ='mse', max_features = "auto")
etr_model = etr.fit(trainingData)
pred = etr_model.transform(testData)
eff_etr = effectivenessMeasuresClassifier(pred)

### ENSAMBLE CLASSIFICATOR - VOTING CLASSIFIER

In [ ]:
from sklearn.ensemble import VotingClassifier

lr_model = linear_model.LinearClassifier()
dc_model = DecisionTreeClassifier()
rf_model = RandomForestClassifier()
estimators = [('lr', lr_model), ('dc', dc_model), ('rf', rf_model)]
vc = VotingClassifier(estimators)
vc_model = vc.fit(trainingData)
pred = vc_model.transform(testData)
vc_etr = effectivenessMeasuresClassifier(pred)

### ENSAMBLE CLASSIFICATOR - STACKING CLASSIFIER

In [ ]:
from sklearn.ensemble import StackingClassifier

lr_model = linear_model.LinearClassifier()
dc_model = DecisionTreeClassifier()
rf_model = RandomForestClassifier()
estimators = [('lr', lr_model), ('dc', dc_model), ('rf', rf_model)]
sc = StackingClassifier(estimators)
sc_model = sc.fit(trainingData)
pred = sc_model.transform(testData)
sc_etr = effectivenessMeasuresClassifier(pred)

## K-KROTNA WALIDAJA KRZYŻOWA

In [ ]:
from random import randrange


def cross_validation_split(data, folds):
    dataset = data.copy().to_numpy()
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / folds)
    for i in range(folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [ ]:
# TO DO STWORZYC FUNKCJE GENERUJACA WYNIKI

In [ ]:
# TO DO: optymalizacja parametrow klasyfikatorow

In [ ]:
import seaborn as sns
plt.bar(trainingData.columns, etr.feature_importance)
plt.xticks(rotation=40)
plt.xlabel('Feature Labels')
plt.ylabel('Feature Importances')
plt.title('Comparison of different Feature Importances')
plt.show()

figure(figsize=(8, 6), dpi=80)

corrMatrix = data.corr()
sns.heatmap(corrMatrix, annot=True)
plt.xlabel('Feature Labels')
plt.ylabel('Feature Importances')
plt.title('Comparison of different Feature Importances')
plt.show()

plt.bar(data.columns[:11], corrMatrix["cnt"][:11])
plt.xticks(rotation=40)
plt.xlabel('Feature Labels')
plt.ylabel('Feature Importances')
plt.title('Comparison of different Feature Importances')
plt.show()

In [ ]:
data = read_data()

size_before_opt = np.shape(data)[1] - 1
print("Ilosc wymairów zmiennych zależnych przed optymalizacją: ", size_before_opt)
data.drop('season', inplace=True, axis=1)
data.drop('mnth', inplace=True, axis=1)
data.drop('holiday', inplace=True, axis=1)
data.drop('weathersit', inplace=True, axis=1)
data.drop('temp', inplace=True, axis=1)
data.drop('hum', inplace=True, axis=1)
data.drop('windspeed', inplace=True, axis=1)
size_after_opt = np.shape(data)[1] - 1
print("Ilosc wymairów zmiennych zależnych po optymalizacji: ",size_after_opt)

x_train, x_test, y_train, y_test = splitOfData(data, 0.20)

In [ ]:
# TO DO: wyniki

In [ ]:
from prettytable import PrettyTable

def add_row(table, result, model):
    avg_result = [sum(x) / len(x) for x in zip(*result)]
    table.add_row([model, round(avg_result[0],5), round(avg_result[1],5), round(avg_result[2],5)])

def createSummaryTable(summary_table, results):
    add_row(summary_table, results[0], "Linear Regression")
    add_row(summary_table, results[1], "Polynominal Regression")
    add_row(summary_table, results[2], "Decision Tree Regression")
    add_row(summary_table, results[3], "Random Forrest Regression")
    add_row(summary_table, results[4], "Voting Regressor")
    add_row(summary_table, results[5], "Stacking Regressor")

def createSummary(k_fold = 0, grid_search_optimalization = False):
    summary_table = PrettyTable(['model', 'MSE', 'r2', 'Experience Variance'])
    if (k_fold < 2):
        results = prepare_result(grid_search_optimalization)
        createSummaryTable(summary_table, results)
        print("Summary table for result of regression models:\n", summary_table, "\n\n")
    else:
        results = prepare_result_with_k_fold(k_fold, grid_search_optimalization)
        createSummaryTable(summary_table, results)
        print("Summary table for result of regression models [K_fold: k =",k_fold,"]:\n", summary_table, "\n\n")

In [ ]:
createSummary()
createSummary(k_fold = 2)
createSummary(k_fold = 5)
createSummary(k_fold = 10)